In [1]:
import requests
import pandas as pd

In [2]:
url = 'https://cov-spectrum.ethz.ch/api/resource/sample2'
params = {'country': 'Switzerland',
          'fields': 'date,region,country,division,ageGroup,sex,hospitalized,deceased,pangolinLineage'}
r = requests.get(url, params)
r.status_code

200

In [3]:
df = pd.read_json(r.content)
# Unroll the count compression, one row per sample
df = df.loc[df.index.repeat(df['count'])]
# Todo: Surveillance vs Non-Surveillance
seq = df
seq = seq.set_index('date')
seq.head()

,division,hospitalized,country,deceased,pangolinLineage,sex,ageGroup,region,count
date,,,,,,,,,
2020-10-14,Basel-Stadt,NaN,Switzerland,NaN,B.1.258,Male,None,Europe,2
2020-10-14,Basel-Stadt,NaN,Switzerland,NaN,B.1.258,Male,None,Europe,2
2021-03-13,Solothurn,0.0,Switzerland,0.0,B.1.1.7,Female,70-79,Europe,1
2021-04-02,Aargau,0.0,Switzerland,0.0,B.1.1.7,Female,50-59,Europe,2
2021-04-02,Aargau,0.0,Switzerland,0.0,B.1.1.7,Female,50-59,Europe,2


In [4]:
# Sequences per calendar week (week beginning and including Monday)
seq_count_by_week = pd.DataFrame(seq.resample(
    'W-MON', label='left', closed='left').country.count())
seq_count_by_week.rename(columns={'country': 'sequences'}, inplace=True)
seq_count_by_week.index.rename('start_of_cw', inplace=True)
seq_count_by_week = seq_count_by_week.assign(
    calendar_week=lambda x: x.index.weekofyear)
seq_count_by_week

<ipython-input-4-36ba90296dbe>:7: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  calendar_week=lambda x: x.index.weekofyear)


,sequences,calendar_week
start_of_cw,,
2020-02-24,35,9
2020-03-02,92,10
2020-03-09,347,11
2020-03-16,492,12
2020-03-23,207,13
...,...,...
2021-05-10,1227,19
2021-05-17,1191,20
2021-05-24,858,21


In [5]:
# Sequences per calendar month (labelled by last day of month)
seq_count_by_week = pd.DataFrame(seq.resample('M').country.count())
seq_count_by_week.rename(columns={'country': 'sequences'}, inplace=True)
seq_count_by_week.index.rename('end_of_month', inplace=True)
seq_count_by_week = seq_count_by_week.assign(month=lambda x: x.index.month)
seq_count_by_week

,sequences,month
end_of_month,,
2020-02-29,29,2
2020-03-31,1211,3
2020-04-30,403,4
2020-05-31,84,5
2020-06-30,96,6
2020-07-31,340,7
2020-08-31,507,8
2020-09-30,423,9
2020-10-31,1778,10


In [6]:
# # %%
# # Plots
# # 1. Table of sequences by calendar week

# # %%
# df.columns
# # %%
# df.head()

# # %%
# df.pangolinLineage.value_counts()[0:60]
# # %%
# df.dtypes
# # %%
# df.pangolinLineage.value_counts()['AY.1']
# # %%
